## Script for creating training sample
### Johan Spanners images

1) you need to flatten Johans file structure  
2) Do it on his drive in a new folder called; flatten_folder  
3) Rename each image acording to a new ID; JS + N_i  (JS = Johan Spanner)
4) Draw 5000 random images  
5) Save in new folder: sample_5000  
5) crate csv frame - that is csv with column names and row index  
6) column names = features  
7) row index = image ID  
8) codebook  
9) Page in code book showing the order of the column names so they don't have to scroll 

to mount external harddriv:
> sudo mkdir /mnt/f # f is just random letter for the mounting point  
> sudo mount -t drvfs d: /mnt/f # f is the letter choosen above, d is the drive name

Now you can access his files at location: **/mnt/f/'October Johan Archive'/Spanner**

You then have the folders Published, Raw, Submitted.

# Code

In [119]:
import os
import cv2
from iptcinfo3 import IPTCInfo
from PIL import Image
from PIL.TiffTags import TAGS # til raw
import rawpy # Måske bedre til raw 
import re
import zipfile
import shutil
import numpy as np
import time
import matplotlib.pyplot as plt

In [2]:
# external_dir =  "/mnt/f/OctoberJohanArchive"
external_dir =  "/mnt/f/OctoberJohanArchiveTest" # only the published images
flatten_dir =  "/mnt/f/OctoberJohanArchiveFlatten"

In [3]:
def count_files(external_dir):
    
    """Counts files in folder tree. Sanity check, and fits in the function flatten_dirs()"""
    
    count = 0
    
    for root, dirs, files in os.walk(external_dir):
        for name in files:
            count += 1
        
    return(count)

In [4]:
def mk_flatten_dir(flatten_dir):
    
    """Creates a new folder which is to be the location of the flattened folder tree. fits in the function flatten_dirs()"""
    
    if not os.path.exists(flatten_dir):
        os.mkdir(flatten_dir) # makes new dir                     
        
    # Or REPLACE old if exist
    else:
        shutil.rmtree(flatten_dir) # delets dir and content.. FOREVER!
        os.mkdir(flatten_dir) # makes new dir


In [5]:
# def infoIPTC() 
#     """Takes infomation from the image path string, and add it to the IPTC. Ex, wheter the is is raw, edit or puplished"""

In [6]:
def flatten_dirs(external_dir, flatten_dir):
    
    time_start = time.time() # start time
    mk_flatten_dir(flatten_dir) # creates the new folder
    files_count = count_files(external_dir) # counts all files to be moved
    counter = 0 # counter for all iterations
    count_move = 0 # counter for files moved and image ID
    count_exceptions = 0 # count files not being move, due to them not being images.
    ID_front = "JS" # Johan Spanner
    
    print(f'\nstarts moving files from: {external_dir}\nto new shared parent dir: {flatten_dir}...\n')
    
    for root, dirs, files in os.walk(external_dir):
        
        for name in files: 
            
            counter += 1
            print(f'Handling file {counter} of {files_count}', end= '\r')
                        
            old_path = os.path.join(root, name) #all the info for the IPTC should be in in old_path name
            
            #store info for IPTC
            old_path.split("/") # list of strings
            
            try: # you only wnat to move actual images
                
                # verify that is an image
                img = Image.open(old_path)  # open the image file
                img.verify()  # verify that it is a good image, without decoding it.. quite fast
                img.close()
                
                # new path
                count_move += 1
                new_path = flatten_dir + "/" + ID_front + str(count_move) + ".jpeg" # path and image ID - could also put the ID in the IPTC now you are at it.
                # seems you can call all relevant image types .jpeg and still open them - even .tif
                
                # Moving day..
                shutil.copyfile(old_path, new_path) # x6 slower then move.. IPTC moves to
#                 shutil.move(old_path, new_path)# will delete the original path. Do IPTC also move? 
#                 shutil.copy(old_path, new_path) # error for both copy and copy2

            except Exception as e: # Non images you just pass
        
                count_exceptions += 1
                pass
                
    # Logging time it takes to complete operation..
    time_end = time.time()
    time_run = round((time_end - time_start)/60,2)
                    
    print(f'\n\nDone moving files to parent dir: {flatten_dir}...')

    print(f'File(s) moved: {counter-count_exceptions} out of {files_count} files')
    print(f'File(s) not moved (not images): {count_exceptions} out of {files_count} files')
    
    print(f'\nALL DONE!\nThe operation took {time_run} minutes')
    


In [7]:
flatten_dirs(external_dir, flatten_dir)


starts moving files from: /mnt/f/OctoberJohanArchiveTest
to new shared parent dir: /mnt/f/OctoberJohanArchiveFlatten...

Handling file 191 of 191

Done moving files to parent dir: /mnt/f/OctoberJohanArchiveFlatten...
File(s) moved: 188 out of 191 files
File(s) not moved (not images): 3 out of 191 files

ALL DONE!
The operation took 0.43 minutes


In [8]:
test_img = "/mnt/f/OctoberJohanArchive/Spanner/Submitted/2003/031703Jylleditfirstnight/A-edit/DSC_0148.tif"

test_img.split("/")#[5]

['',
 'mnt',
 'f',
 'OctoberJohanArchive',
 'Spanner',
 'Submitted',
 '2003',
 '031703Jylleditfirstnight',
 'A-edit',
 'DSC_0148.tif']

In [9]:
external_dir

'/mnt/f/OctoberJohanArchiveTest'

## Check IPTC in moved images.

'date created' = date   
'time created' = time  
'copyright notice' = Johan spanner  
'custom1' = raw, submitted, pulished  

In [10]:
dict_keys = ['object name', 
             'edit status', 
             'editorial update', 
             'urgency', 
             'subject reference', 
             'category', 
             'supplemental category', 
             'fixture identifier', 
             'keywords', 
             'content location code', 
             'content location name', 
             'release date', 
             'release time', 
             'expiration date', 
             'expiration time', 
             'special instructions', 
             'action advised', 
             'reference service', 
             'reference date', 
             'reference number', 
             'date created', 
             'time created', 
             'digital creation date', 
             'digital creation time', 
             'originating program', 
             'program version', 
             'object cycle', 
             'by-line', 
             'by-line title', 
             'city', 
             'sub-location', 
             'province/state', 
             'country/primary location code', 
             'country/primary location name', 
             'original transmission reference', 
             'headline', 
             'credit', 
             'source', 
             'copyright notice', 
             'contact', 
             'caption/abstract', 
             'local caption', 
             'writer/editor', 
             'image type', 
             'image orientation', 
             'language identifier', 
             'custom1', 
             'custom2', 
             'custom3', 
             'custom4', 
             'custom5', 
             'custom6', 
             'custom7', 
             'custom8', 
             'custom9', 
             'custom10', 
             'custom11', 
             'custom12', 
             'custom13', 
             'custom14', 
             'custom15', 
             'custom16', 
             'custom17', 
             'custom18', 
             'custom19', 
             'custom20'] # surely you can get this list from somewhere...
# save as a pickle so you don't have to look at this shit.

In [11]:
def get_IPTC_data(image_path):
    info = IPTCInfo(image_path, force=True, inp_charset='utf8')
    
    keys = ['custom1','copyright notice','date created','time created'] #or by-line for name?
    strings = ['submitted','Spanner',' a date', 'some time'] # submitted and copyright notice you get from path, date and time created you get from..., 
    
    for j,i in enumerate(keys):
        if info[i] == None or len(info[i]) < 1:
                info[i] = strings[j] # if you sort the string list first, you could use enumarate and strings[j]
                
    #--------------------------------------------
    
    for i in dict_keys:
        if info[i] != None:
            if len(info[i]) > 0: 
                print(f'key: {i}, info: {info[i]}\n')


# construction:

In [137]:
# submitted
test_imgSubTIF = "/mnt/f/OctoberJohanArchiveTestOld/Spanner/Submitted/DSC_0128.tif"
test_imgSubJPG = "/mnt/f/OctoberJohanArchiveTestOld/Spanner/Submitted/DSC_0906.jpg"

# raw
test_imgRawNEF = "/mnt/f/OctoberJohanArchiveTestOld/Spanner/Raw/DSC_0001.NEF"
test_imgRawCR2 = "/mnt/f/OctoberJohanArchiveTestOld/Spanner/Raw/IMG_0001.CR2"
test_imgRawJPG = "/mnt/f/OctoberJohanArchiveTestOld/Spanner/Raw/P3300001.JPG"



In [102]:
def get_IPTC_data(image_path):
    
    info = IPTCInfo(image_path, force=True) #, inp_charset='utf8')
    image = Image.open(image_path)
    
    # should be somewhere else..
#     image = image.convert('RGB')
#     image.save(image_path) 
    #---------------------------
#     info = IPTCInfo(image_path, force=True, inp_charset='utf8')

    
    try:
        txt = str(image.info['exif']) # exif are mostly JPEG - what if I convert myself?
    
        r = '[\d]{4}:[\d]{2}:[\d]{2} [\d]{2}:[\d]{2}:[\d]{2}'# time and, date. but is this robust? we will find out.
    
        date, time = re.search(r, txt).group().replace(':','').split(' ')
        
    
    except:
        date = 'D'
        time = 'T'
        pass
    
    keys = ['custom1','copyright notice','date created','time created'] #or by-line for name?
    strings = ['submitted','Spanner', date, time] # submitted and copyright notice you get from path, date and time created you get from..., 
    
    for j,i in enumerate(keys):
        if info[i] == None or len(info[i]) < 1:
                info[i] = strings[j]
                
#Print for test--------------------------------------------
    
    for i in dict_keys:
        if info[i] != None:
            if len(info[i]) > 0: 
                print(f'key: {i}, info: {info[i]}\n')


# Really construction!
but also solutions.

In [166]:
# works for CR2 and NEF
raw = rawpy.imread(test_imgRawCR2)
meta_string = str(raw.extract_thumb())

In [ ]:
# works for tiff
img = Image.open(test_imgSubTIF)
meta_string = str(img.tag.tagdata) # seems, there are slightly different timestamp [2 hours dif - time zone tingy?], but one is correctly formatted.

In [188]:
# works for jpeg
img = Image.open(test_imgRawJPG)
meta_string = str(img.info['exif'])

# REST:

In [ ]:
# works for tiff
with Image.open(test_imgSubTIF) as img:
    meta_dict = {TAGS[key] : img.tag[key] for key in img.tag.keys()}

In [167]:
img = Image.open(test_imgSubTIF)
str(img.tag.tagdata)

In [178]:
for key in img.tag.keys():
    print(key) # men det er næppe det samme universielt...

256
257
258
259
262
34377
273
277
278
279
282
283
284
296
34665
305
306
34675
700
254


In [159]:
img.tag[34377]

(b'8BIM\x04%\x00\x00\x00\x00\x00\x10\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x008BIM\x03\xea\x00\x00\x00\x00\x1d\xad<?xml version="1.0" encoding="UTF-8"?>\n<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">\n<plist version="1.0">\n<dict>\n\t<key>com.apple.print.PageFormat.PMHorizontalRes</key>\n\t<dict>\n\t\t<key>com.apple.print.ticket.creator</key>\n\t\t<string>com.apple.printingmanager</string>\n\t\t<key>com.apple.print.ticket.itemArray</key>\n\t\t<array>\n\t\t\t<dict>\n\t\t\t\t<key>com.apple.print.PageFormat.PMHorizontalRes</key>\n\t\t\t\t<real>72</real>\n\t\t\t\t<key>com.apple.print.ticket.client</key>\n\t\t\t\t<string>com.apple.printingmanager</string>\n\t\t\t\t<key>com.apple.print.ticket.modDate</key>\n\t\t\t\t<date>2003-03-22T10:07:41Z</date>\n\t\t\t\t<key>com.apple.print.ticket.stateFlag</key>\n\t\t\t\t<integer>0</integer>\n\t\t\t</dict>\n\t\t</array>\n\t</dict>\n\t<key>com.apple.print.PageFormat.P

In [185]:
str(img.tag.tagdata)

'{254: b\'\\x00\\x00\\x00\\x00\', 256: b\'\\xd0\\x07\', 257: b\' \\x05\', 258: b\'\\x10\\x00\\x10\\x00\\x10\\x00\', 259: b\'\\x01\\x00\', 262: b\'\\x02\\x00\', 273: b\'\\xb0F\\x00\\x00\', 277: b\'\\x03\\x00\', 278: b\' \\x05\', 279: b\'\\x00<\\xf0\\x00\', 282: b"\\xc0\\xc6-\\x00\\x10\'\\x00\\x00", 283: b"\\xc0\\xc6-\\x00\\x10\'\\x00\\x00", 284: b\'\\x01\\x00\', 296: b\'\\x02\\x00\', 305: b\'Adobe Photoshop 7.0\\x00\', 306: b\'2003:03:22 12:16:17\\x00\', 700: b\'<?xpacket begin=\\\'\\xef\\xbb\\xbf\\\' id=\\\'W5M0MpCehiHzreSzNTczkc9d\\\'?>\\n<?adobe-xap-filters esc="CR"?>\\n<x:xapmeta xmlns:x=\\\'adobe:ns:meta/\\\' x:xaptk=\\\'XMP toolkit 2.8.2-33, framework 1.5\\\'>\\n<rdf:RDF xmlns:rdf=\\\'http://www.w3.org/1999/02/22-rdf-syntax-ns#\\\' xmlns:iX=\\\'http://ns.adobe.com/iX/1.0/\\\'>\\n\\n <rdf:Description about=\\\'uuid:3c9ad561-5df2-11d7-818b-96dac26cff60\\\'\\n  xmlns:xapMM=\\\'http://ns.adobe.com/xap/1.0/mm/\\\'>\\n  <xapMM:DocumentID>adobe:docid:photoshop:212b550f-5df1-11d7-818b-96d

In [ ]:
with Image.open(test_imgSubTIF) as img:
    meta_dict = {TAGS[key] : img.tag[key] for key in img.tag.keys()}

In [139]:
with Image.open(test_imgSubTIF) as img:
    meta_dict = {TAGS[key] : img.tag[key] for key in img.tag.keys()}

In [140]:
meta_dict

{'ImageWidth': (2000,),
 'ImageLength': (1312,),
 'BitsPerSample': (16, 16, 16),
 'Compression': (1,),
 'PhotometricInterpretation': (2,),
 'PhotoshopInfo': (b'8BIM\x04%\x00\x00\x00\x00\x00\x10\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x008BIM\x03\xea\x00\x00\x00\x00\x1d\xad<?xml version="1.0" encoding="UTF-8"?>\n<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">\n<plist version="1.0">\n<dict>\n\t<key>com.apple.print.PageFormat.PMHorizontalRes</key>\n\t<dict>\n\t\t<key>com.apple.print.ticket.creator</key>\n\t\t<string>com.apple.printingmanager</string>\n\t\t<key>com.apple.print.ticket.itemArray</key>\n\t\t<array>\n\t\t\t<dict>\n\t\t\t\t<key>com.apple.print.PageFormat.PMHorizontalRes</key>\n\t\t\t\t<real>72</real>\n\t\t\t\t<key>com.apple.print.ticket.client</key>\n\t\t\t\t<string>com.apple.printingmanager</string>\n\t\t\t\t<key>com.apple.print.ticket.modDate</key>\n\t\t\t\t<date>2003-03-22T10:07:41Z</date>\n\

In [90]:
get_IPTC_data(test_imgRawNjpg)

Marker scan hit start of image data


key: date created, info: D

key: time created, info: T

key: copyright notice, info: Spanner

key: custom1, info: submitted



In [48]:
info = IPTCInfo(test_imgRaw2, force=True, inp_charset='utf8')

Marker scan hit start of image data


In [41]:
image.convert?

Object `image.convert` not found.


In [50]:
# Open the image and access the metadate. Use regex to extract date:
image = Image.open(test_imgRaw) 
txt = str(image.info['exif']) # exif are mostly JPEG - what is I convert myself?

r = '[\d]{4}:[\d]{2}:[\d]{2} [\d]{2}:[\d]{2}:[\d]{2}'# time and, date. but is this robust? we will find out.

data, time = re.search(r, txt).group().replace(':','').split(' ')

# data, time = date_timedata, time

print(f'date of image: {date}\ntime of image: {time}')

date of image: 20030322
time of image: 171903


In [40]:
txt

'b\'Exif\\x00\\x00MM\\x00*\\x00\\x00\\x00\\x08\\x00\\x07\\x01\\x12\\x00\\x03\\x00\\x00\\x00\\x01\\x00\\x01\\x00\\x00\\x01\\x1a\\x00\\x05\\x00\\x00\\x00\\x01\\x00\\x00\\x00b\\x01\\x1b\\x00\\x05\\x00\\x00\\x00\\x01\\x00\\x00\\x00j\\x01(\\x00\\x03\\x00\\x00\\x00\\x01\\x00\\x02\\x00\\x00\\x011\\x00\\x02\\x00\\x00\\x00\\x14\\x00\\x00\\x00r\\x012\\x00\\x02\\x00\\x00\\x00\\x14\\x00\\x00\\x00\\x86\\x87i\\x00\\x04\\x00\\x00\\x00\\x01\\x00\\x00\\x00\\x9c\\x00\\x00\\x00\\xc8\\x00\\x00\\x00\\xc8\\x00\\x00\\x00\\x01\\x00\\x00\\x00\\xc8\\x00\\x00\\x00\\x01Adobe Photoshop 7.0\\x002003:03:22 17:19:03\\x00\\x00\\x00\\x00\\x03\\xa0\\x01\\x00\\x03\\x00\\x00\\x00\\x01\\xff\\xff\\x00\\x00\\xa0\\x02\\x00\\x04\\x00\\x00\\x00\\x01\\x00\\x00\\x02\\xfa\\xa0\\x03\\x00\\x04\\x00\\x00\\x00\\x01\\x00\\x00\\x01\\xf4\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x06\\x01\\x03\\x00\\x03\\x00\\x00\\x00\\x01\\x00\\x06\\x00\\x00\\x01\\x1a\\x00\\x05\\x00\\x00\\x00\\x01\\x00\\x00\\x01\\x16\\x01\\x1b\\x00\\x05\\x00\\x00\\x00\\x01\\x0

In [52]:
get_IPTC_data(test_imgRaw)

key: date created, info: 20030322

key: time created, info: 171903

key: copyright notice, info: Spanner

key: custom1, info: submitted



In [49]:
get_IPTC_data(test_imgPub)

key: date created, info: 20090101

key: time created, info: 113111+0300

key: by-line, info: Johan Spanner

key: city, info: Baghdad

key: country/primary location name, info: Iraq

key: headline, info: Green Zone handover

key: credit, info: Johan Spanner for The New York Times

key: copyright notice, info: Johan Spanner

 he ceremony was attended by Iraqi Minister of Defense, National security, top Iraqi and some American army colonels, Iraqi police Generals and the American Embassy and the head of the Baghdad provincial Council dedicated to the handover of the Green Zone. Iraqi national anthem was played several times. Minister of Defense, Brigadier General Emad, commander of Baghdad Brigade which will control the Green Zone, and Colonel Ferrari who commanded the American unit in the Green Zone, gave speeches.

key: custom1, info: submitted

